In [2]:
import numpy as np
from scipy.special import jv
from scipy.optimize import newton
import imageio as iio
import os
import math

# Pixel size
lamda = 0.66e-6  # 波长单位为微米
w0 = 900e-6
pix_size = 9.2e-6

# Parameters
mx = 1920
my = 1152

# Create base coordinate grids
x_base = np.linspace(-mx * pix_size / 2, mx * pix_size / 2, mx)
y_base = np.linspace(-my * pix_size / 2, my * pix_size / 2, my)
X_base, Y_base = np.meshgrid(x_base, y_base)

# 光束参数
zr = np.pi * w0**2 / lamda
k0 = 2 * np.pi / lamda

# Grating phase
grating_period = pix_size * 5
gra = (2 * np.pi * X_base / grating_period) % (2 * np.pi)

# Function to generate phase pattern with variable xe, ye, z shifts
def generate_GS(X, Y, z, xe=0, ye=0):
    # 距离计算
    r_sq = (X - xe)**2 + (Y - ye)**2  # (x - xe)^2 + (y - ye)^2
    if z == 0:
        Rz = np.inf  # 避免 z = 0 时除以零的情况
    else:
        Rz = z * (1 + (zr / z)**2)  # 曲率半径
    
    wz = w0 * np.sqrt(1 + (z / zr)**2)  # 光束宽度
    
    # 振幅项
    amplitude = np.sqrt(2 / (np.pi * wz**2)) * np.exp(-r_sq / wz**2)
    
    # 相位项
    phase = np.exp(-1j * k0 * z) * np.exp(-1j * k0 * r_sq / (2 * Rz)) * np.exp(1j * np.arctan(z / zr))
    
    # 最终结果
    return amplitude * phase

# Method F
C = 0.5818

def faeq(f_a_ref, a_ref):
    global C
    return jv(1, f_a_ref) - C * a_ref

def find_f_a_ref(a_ref):
    return newton(faeq, np.ones_like(a_ref) * 1, args=(a_ref,))

# Output directory
output_dir = r'C:\Users\wkx\Desktop\wkx1\程序\python\相图'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Generate images for xe values from -50*pix_size to 50*pix_size
xe_values = np.arange(-50 * pix_size, 51 * pix_size, 5 * pix_size)
ye = 0  # Keep ye constant
z = 4e-6  # Keep z constant for this part

for i, xe in enumerate(xe_values):
    X1 = X_base  # 不需要修改X
    Y1 = Y_base  # 不需要修改Y
    u1 = generate_GS(X1, Y1, z, xe=xe, ye=ye)
    AM1 = np.abs(u1) / np.max(np.abs(u1))
    phi1 = np.angle(u1) + gra - np.pi
    
    fa1 = find_f_a_ref(AM1)
    h1 = fa1 * np.sin(phi1)
    h1 = np.mod(h1 + np.min(h1), 2 * np.pi)

    gh1 = (h1 * 255 / (2 * np.pi)).astype(np.uint8)
    
    gs_image_path = os.path.join(output_dir, f'gs_phase_shift_xe_{i+1}.bmp')
    iio.imwrite(gs_image_path, gh1)

print(f"First set of images (xe variation) saved to {output_dir}")

# Generate images for z values from 0 to 10^9, xe = 0*pix_size
z_values = np.arange(0, 1e9 + 1e8, 1e8)
xe = 0  # Set xe constant for this part

for i, z in enumerate(z_values):
    X1 = X_base  # 不需要修改X
    Y1 = Y_base  # 不需要修改Y
    u1 = generate_GS(X1, Y1, z, xe=xe, ye=ye)
    AM1 = np.abs(u1) / np.max(np.abs(u1))
    phi1 = np.angle(u1) + gra - np.pi
    
    fa1 = find_f_a_ref(AM1)
    h1 = fa1 * np.sin(phi1)
    h1 = np.mod(h1 + np.min(h1), 2 * np.pi)

    gh1 = (h1 * 255 / (2 * np.pi)).astype(np.uint8)
    
    gs_image_path = os.path.join(output_dir, f'gs_phase_shift_z_{i+1}.bmp')
    iio.imwrite(gs_image_path, gh1)

print(f"Second set of images (z variation) saved to {output_dir}")


First set of images (xe variation) saved to C:\Users\wkx\Desktop\wkx1\程序\python\相图
Second set of images (z variation) saved to C:\Users\wkx\Desktop\wkx1\程序\python\相图
